# Riduzione DFA con prefix-tree & consistent cover


In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent / "src"))
Path("../outputs").mkdir(parents=True, exist_ok=True)
from generate_sets import derive_sets
from tree_builder import build_prefix_tree
from dfa_examples import build_dfa

states = {"q0", "q1", "q2"}
alphabet = {"a", "b"}
transitions = {
    ("q0", "a"): "q1",
    ("q1", "b"): "q2",
    ("q2", "a"): "q0",
}
initial_state = "q0"
final_states = {"q2"}

dfa = build_dfa(states, alphabet, transitions, initial_state, final_states)
dfa.to_png("../outputs/custom_dfa")

A, G, N = derive_sets(dfa, max_len=4)
tree = build_prefix_tree(A, G, N, alphabet=dfa.Sigma)

# Esporta per Graphviz
tree.export_dot(Path("../outputs/prefix_tree.dot"))


DFA salvato in result\..\outputs\custom_dfa.png
[tree_builder] salvato .dot in ..\outputs\prefix_tree.dot


## Import delle funzioni

In [3]:
# Cella 1
from tree_builder import build_prefix_tree
from reduction import (
    build_initial_cover,
    refine_cover,
    build_reduced_dfa,
    export_dfa_dot,
)

print("Moduli importati correttamente!")

# piccola demo con il DFA di esempio
from dfa_examples import dfa_simple
from generate_sets import derive_sets


Moduli importati correttamente!


# Pipeline 

In [ ]:
# Cella 2 – Generazione insiemi A, G, N dal DFA originale
dfa = build_dfa(states, alphabet, transitions, initial_state, final_states)
dfa.to_png("../outputs/initial_dfa")
A, G, N = derive_sets(dfa, max_len=4)    # analizza stringhe ≤4
print("A =", A)
print("G =", G)
print("N =", N)


DFA salvato in result\..\outputs\initial_dfa.png
A = {'ab'}
G = {'a', 'abaa', 'aba'}
N = {'bbab', 'abab', 'bba', 'bbbb', 'baaa', 'aaab', 'abb', 'baa', 'bbb', 'baba', 'bb', 'bbaa', 'aaaa', 'baab', 'aaa', 'aa', 'aabb', 'aab', 'ba', 'aaba', 'abbb', 'b', 'bab', 'abba', 'bbba', 'babb'}


In [12]:
# Cella 3 – Costruzione prefix-tree (e file .dot opzionale)
tree = build_prefix_tree(A, G, N, alphabet=dfa.Sigma)
tree.export_dot("../outputs/prefix_tree.dot")   # commenta se non ti serve
print("Nodi totali nel tree:", sum(1 for _ in tree.iter_nodes()))


[tree_builder] salvato .dot in ../outputs/prefix_tree.dot
Nodi totali nel tree: 31


In [13]:
!dot -Tpng ..\outputs\prefix_tree.dot -o ..\outputs\prefix_tree.png                

In [6]:
# Cella 4 – Cover iniziale Π e raffinamento in cover coerente
cover0 = build_initial_cover(tree)
cover_final = refine_cover(tree, cover0)

print("Celle iniziali:", len(cover0))
print("Celle finali  :", len(cover_final))


Celle iniziali: 3
Celle finali  : 3


In [ ]:
# Cella 5 – Costruzione DFA ridotto + esportazione .dot
dfa_red = build_reduced_dfa(tree, cover_final)
export_dfa_dot(dfa_red, "../outputs/reduced_dfa.dot")   # commenta se non ti serve
dfa_red.to_png("../outputs/reduced_dfa.png")  # commenta se non ti serve

print("Stati DFA ridotto:", len(dfa_red.Q))
print("Stato iniziale   :", dfa_red.q0)
print("Stati finali     :", dfa_red.F)

[reduction] .dot salvato in ../outputs/reduced_dfa.dot
DFA salvato in result\..\outputs\reduced_dfa.png.png
Stati DFA ridotto: 3
Stato iniziale   : q2
Stati finali     : {'q2'}


### Uso del DFA ridotto


In [10]:
# Cella 6 – Esempio di simulazione sul DFA ridotto
test_word = "abba"
q = dfa_red.q0
for ch in test_word:
    q = dfa_red.next(q, ch)
    print(f"δ({q}, {ch}) -> {q}")
print("Accettata?" , q in dfa_red.F)


δ(q1, a) -> q1
δ(None, b) -> None
δ(None, b) -> None
δ(None, a) -> None
Accettata? False
